# scratch work

In [15]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit.utils.states import SystemState
from jorbit.data.constants import INV_SPEED_OF_LIGHT
from jorbit.integrators import ias15_evolve, initialize_ias15_integrator_state

t = Time("2024-12-01 00:00")

In [16]:
from jorbit.particle import Particle
from jorbit.system import System

obj = Horizons(id="274301", location="@0", epochs=t.tdb.jd)
vecs = obj.vectors(refplane="earth")

x = jnp.array([vecs["x"][0], vecs["y"][0], vecs["z"][0]])
v = jnp.array([vecs["vx"][0], vecs["vy"][0], vecs["vz"][0]])

p = Particle(x=x, v=v, time=t, name="wiki")
system = System(particles=[p], acceleration_func="newtonian planets")

print(system.acceleration_func(system.state))

system

[[ 2.61030935e-05 -2.90119924e-05 -8.52860259e-06]]


*************
jorbit System
 time: 2460645.50080073
 particles: [Particle: wiki]
*************

In [17]:
from jorbit.observation import Observations

obj = Horizons(id="274301", location="695", epochs=t.jd)
eph = obj.ephemerides(extra_precision=True, quantities="1")
coord = SkyCoord(eph["RA"][0], eph["DEC"][0], unit=(u.deg, u.deg), frame="icrs")
print(coord)

obs = Observations(
    observed_coordinates=coord,
    times=t,
    observatories="695@399",
    astrometric_uncertainties=1 * u.arcsec,
    verbose=True,
)
obs

<SkyCoord (ICRS): (ra, dec) in deg
    (152.01007338, 5.09857717)>


Observations with 1 set(s) of observations

In [18]:
@jax.jit
def on_sky(
    state: SystemState,
    acc_func,
    observer_position,
):
    a0 = acc_func(state)
    initial_integrator_state = initialize_ias15_integrator_state(a0)

    def scan_func(carry, scan_over):
        xz = carry
        earth_distance = jnp.linalg.norm(xz - observer_position)
        light_travel_time = earth_distance * INV_SPEED_OF_LIGHT

        positions, final_system_state, final_integrator_state = ias15_evolve(
            state,
            acc_func,
            jnp.array([state.time - light_travel_time]),
            initial_integrator_state,
            n_steps=5,
        )

        return final_system_state.positions, None

    xz = jax.lax.scan(scan_func, state.positions, None, length=2)[0]

    X = xz - observer_position
    calc_ra = jnp.mod(jnp.arctan2(X[:, 1], X[:, 0]) + 2 * jnp.pi, 2 * jnp.pi)
    calc_dec = jnp.pi / 2 - jnp.arccos(X[:, -1] / jnp.linalg.norm(X, axis=1))
    return calc_ra, calc_dec


a, b = on_sky(system.state, system.acceleration_func, obs.observer_positions[0])
c = SkyCoord(a, b, unit=(u.rad, u.rad))

c.separation(coord).to(u.microarcsecond)

<Angle [1.14999621] uarcsec>

In [19]:
%%timeit

on_sky(system.state, system.acceleration_func, obs.observer_positions[0])[
    0
].block_until_ready()

544 μs ± 2.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [20]:
@jax.jit
def tmp(state):
    a0 = system.acceleration_func(state)
    initial_integrator_state = initialize_ias15_integrator_state(a0)
    return initial_integrator_state


%timeit tmp(system.state).g.p0.block_until_ready()

259 μs ± 100 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit.utils.states import SystemState
from jorbit.data.constants import INV_SPEED_OF_LIGHT
from jorbit.integrators import ias15_evolve, initialize_ias15_integrator_state

t = Time("2024-12-01 00:00")

In [22]:
obj = Horizons(id="274301", location="@0", epochs=t.jd)

vec_earth = obj.vectors(refplane="earth")
vec_ecliptic = obj.vectors(refplane="ecliptic")

elements_earth = obj.elements(refplane="earth")
elements_ecliptic = obj.elements(refplane="ecliptic")

In [23]:
vec_earth

targetname,datetime_jd,datetime_str,H,G,x,y,z,vx,vy,vz,lighttime,range,range_rate
---,d,---,mag,---,AU,AU,AU,AU / d,AU / d,AU / d,d,AU,AU / d
str28,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
274301 Wikipedia (2008 QH24),2460645.5,A.D. 2024-Dec-01 00:00:00.0000,16.98,0.15,-1.783546864189309,1.972824404244611,0.5796727162144323,-0.007524434581651495,-0.005768622712272137,-0.00178665436068034,0.01572076145339102,2.721965467206746,0.0003688611274302079


In [24]:
vec_ecliptic

targetname,datetime_jd,datetime_str,H,G,x,y,z,vx,vy,vz,lighttime,range,range_rate
---,d,---,mag,---,AU,AU,AU,AU / d,AU / d,AU / d,d,AU,AU / d
str28,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
274301 Wikipedia (2008 QH24),2460645.5,A.D. 2024-Dec-01 00:00:00.0000,16.98,0.15,-1.783546864189309,2.040611566933855,-0.2529051616758324,-0.007524434581651495,-0.006003298151579335,0.0006554030090899591,0.01572076145339102,2.721965467206745,0.0003688611274302083


In [25]:
from jorbit.astrometry.transformations import (
    icrs_to_horizons_ecliptic,
    horizons_ecliptic_to_icrs,
)

v1 = jnp.array([vec_earth["x"][0], vec_earth["y"][0], vec_earth["z"][0]])
v2 = jnp.array([vec_ecliptic["x"][0], vec_ecliptic["y"][0], vec_ecliptic["z"][0]])

icrs_to_horizons_ecliptic(v1) - v2, horizons_ecliptic_to_icrs(v2) - v1

(Array([ 0.00000000e+00,  0.00000000e+00, -5.55111512e-17], dtype=float64),
 Array([0., 0., 0.], dtype=float64))

In [26]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit.utils.states import SystemState
from jorbit.data.constants import INV_SPEED_OF_LIGHT
from jorbit.integrators import ias15_evolve, initialize_ias15_integrator_state

t = Time("2024-12-01 00:00")

In [27]:
obj = Horizons(id="274301", location="500@10", epochs=t.jd)
elements_earth_helio = obj.elements(refplane="earth")
elements_ecliptic_helio = obj.elements(refplane="ecliptic")

obj = Horizons(id="274301", location="@0", epochs=t.jd)
elements_earth_bary = obj.elements(refplane="earth")
elements_ecliptic_bary = obj.elements(refplane="ecliptic")


obj = Horizons(id="274301", location="@0", epochs=t.jd)
vec_earth = obj.vectors(refplane="earth")
vec_ecliptic = obj.vectors(refplane="ecliptic")
v1 = jnp.array([vec_earth["x"][0], vec_earth["y"][0], vec_earth["z"][0]])
v2 = jnp.array([vec_ecliptic["x"][0], vec_ecliptic["y"][0], vec_ecliptic["z"][0]])

In [28]:
elements_earth_helio

targetname,datetime_jd,datetime_str,H,G,e,q,incl,Omega,w,Tp_jd,n,M,nu,a,Q,P
---,d,---,mag,---,---,AU,deg,deg,deg,d,deg / d,deg,deg,AU,AU,d
str28,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
274301 Wikipedia (2008 QH24),2460645.5,A.D. 2024-Dec-01 00:00:00.0000,16.98,0.15,0.1477148140497017,2.028905789332561,16.72425227045122,358.653194209116,324.1826533027365,2460034.68598474,0.2683420056828497,163.9070579541506,167.885831600086,2.380547993533797,2.732190197735032,1341.571548158881


In [29]:
elements_earth_bary

targetname,datetime_jd,datetime_str,H,G,e,q,incl,Omega,w,Tp_jd,n,M,nu,a,Q,P
---,d,---,mag,---,---,AU,deg,deg,deg,d,deg / d,deg,deg,AU,AU,d
str28,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
274301 Wikipedia (2008 QH24),2460645.5,A.D. 2024-Dec-01 00:00:00.0000,16.98,0.15,0.1497448681928601,2.021477076373556,16.72132359306627,358.6275689897468,324.8545579746402,2460038.896554522,0.2690394192855276,163.200238708019,167.3990649463254,2.377494708061438,2.733512339749321,1338.093878421352


In [30]:
elements = {
    "a": jnp.array([elements_earth_bary["a"][0]]),
    "ecc": jnp.array([elements_earth_bary["e"][0]]),
    "inc": jnp.array([elements_earth_bary["incl"][0]]),
    "Omega": jnp.array([elements_earth_bary["Omega"][0]]),
    "omega": jnp.array([elements_earth_bary["w"][0]]),
    "nu": jnp.array([elements_earth_bary["nu"][0]]),
}
elements

from jorbit.astrometry.transformations import elements_to_cartesian

c = elements_to_cartesian(**elements)
c

(Array([[-1.78354686,  1.9728244 ,  0.57967272]], dtype=float64),
 Array([[-0.00752443, -0.00576862, -0.00178665]], dtype=float64))

In [31]:
c[0] - v1

Array([[-8.88178420e-16, -1.99840144e-15, -5.55111512e-16]], dtype=float64)

In [32]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit.utils.states import SystemState
from jorbit.data.constants import INV_SPEED_OF_LIGHT
from jorbit.integrators import ias15_evolve, initialize_ias15_integrator_state

t = Time("2024-12-01 00:00")

In [33]:
# refplane = "earth"
refplane = "ecliptic"
obj = Horizons(id="274301", location="@0", epochs=t.jd)
elements_earth_bary = obj.elements(refplane=refplane)

obj = Horizons(id="274301", location="@0", epochs=t.jd)
vec = obj.vectors(refplane=refplane)
x = jnp.array([[vec["x"][0], vec["y"][0], vec["z"][0]]])
v = jnp.array([[vec["vx"][0], vec["vy"][0], vec["vz"][0]]])

In [34]:
elements_earth_bary

targetname,datetime_jd,datetime_str,H,G,e,q,incl,Omega,w,Tp_jd,n,M,nu,a,Q,P
---,d,---,mag,---,---,AU,deg,deg,deg,d,deg / d,deg,deg,AU,AU,d
str28,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
274301 Wikipedia (2008 QH24),2460645.5,A.D. 2024-Dec-01 00:00:00.0000,16.98,0.15,0.1497448681928601,2.021477076373556,6.734028673772495,183.3690911324457,140.194237364336,2460038.896554522,0.2690394192855275,163.200238708019,167.3990649463254,2.377494708061439,2.733512339749321,1338.093878421353


In [35]:
from jorbit.astrometry.transformations import cartesian_to_elements

print(cartesian_to_elements(x, v)["a"] - elements_earth_bary["a"][0])
print(cartesian_to_elements(x, v)["ecc"] - elements_earth_bary["e"][0])
print(cartesian_to_elements(x, v)["inc"] - elements_earth_bary["incl"][0])
print(cartesian_to_elements(x, v)["Omega"] - elements_earth_bary["Omega"][0])
print(cartesian_to_elements(x, v)["omega"] - elements_earth_bary["w"][0])
print(cartesian_to_elements(x, v)["nu"] - elements_earth_bary["nu"][0])

[3.58066021e-10]
[-1.66591796e-10]
[2.13162821e-14]
[-8.52651283e-14]
[1.68317342e-08]
[-1.68317627e-08]


In [36]:
from jorbit.astrometry.transformations import elements_to_cartesian
from jorbit.astrometry.transformations import cartesian_to_elements


# # refplane = "earth"
# refplane = "ecliptic"
# obj = Horizons(id="274301", location="@0", epochs=t.jd)
# elements_earth_bary = obj.elements(refplane=refplane)

obj = Horizons(id="274301", location="@0", epochs=t.jd)
vec = obj.vectors(refplane=refplane)
x = jnp.array(
    [[vec["x"][0], vec["y"][0], vec["z"][0]], [vec["x"][0], vec["y"][0], vec["z"][0]]]
)
v = jnp.array(
    [
        [vec["vx"][0], vec["vy"][0], vec["vz"][0]],
        [vec["vx"][0], vec["vy"][0], vec["vz"][0]],
    ]
)

el = cartesian_to_elements(x, v)
x_new, v_new = elements_to_cartesian(**el)

x_new - x, v_new - v

(Array([[ 2.66453526e-15,  0.00000000e+00, -5.55111512e-16],
        [ 2.66453526e-15,  0.00000000e+00, -5.55111512e-16]],      dtype=float64),
 Array([[-6.93889390e-18,  3.46944695e-18,  2.49366500e-18],
        [-6.93889390e-18,  3.46944695e-18,  2.49366500e-18]],      dtype=float64))